In [1]:
import pandas as pd
from pathlib import Path
import numpy as np

# File to Load
homicide_csv = Path("Resources/homicide_data.csv")

# Read homicide data file and store into Pandas DataFrame
homicide_data = pd.read_csv(homicide_csv, encoding='latin1')

In [2]:
homicide_data.head()

,uid,reported_date,victim_last,victim_first,victim_race,victim_age,victim_sex,city,state,lat,lon,disposition
0,Alb-000001,20100504,GARCIA,JUAN,Hispanic,78,Male,Albuquerque,NM,35.095788,-106.538555,Closed without arrest
1,Alb-000002,20100216,MONTOYA,CAMERON,Hispanic,17,Male,Albuquerque,NM,35.056810,-106.715321,Closed by arrest
2,Alb-000003,20100601,SATTERFIELD,VIVIANA,White,15,Female,Albuquerque,NM,35.086092,-106.695568,Closed without arrest
3,Alb-000004,20100101,MENDIOLA,CARLOS,Hispanic,32,Male,Albuquerque,NM,35.078493,-106.556094,Closed by arrest
4,Alb-000005,20100102,MULA,VIVIAN,White,72,Female,Albuquerque,NM,35.130357,-106.580986,Closed without arrest


Identified cities with reported_date issues. We corrected them using the assumtion there were repeated '1's.

In [3]:
specific_uid1 = homicide_data.loc[homicide_data['uid'] == 'Mia-000649']
specific_uid1

,uid,reported_date,victim_last,victim_first,victim_race,victim_age,victim_sex,city,state,lat,lon,disposition
33584,Mia-000649,201511018,SALAS,LUIS,Hispanic,Unknown,Male,Miami,FL,25.7699,-80.217186,Closed by arrest


In [4]:
specific_uid2 = homicide_data.loc[homicide_data['uid'] == 'Mia-000652']
specific_uid2

,uid,reported_date,victim_last,victim_first,victim_race,victim_age,victim_sex,city,state,lat,lon,disposition
33587,Mia-000652,201511105,BUNCH,GERALD A.,Black,Unknown,Male,Miami,FL,25.826947,-80.202117,Open/No arrest


In [5]:
# Assuming 'uid' is a unique identifier in your DataFrame
chosen_uid1 = 'Mia-000649'
new_reported_date1 = '20151018'

# Locate the specific row with the given 'uid' value and update the 'reported_date'
homicide_data.loc[homicide_data['uid'] == chosen_uid1, 'reported_date'] = new_reported_date1

In [6]:
# Assuming 'uid' is a unique identifier in your DataFrame
chosen_uid2 = 'Mia-000652'
new_reported_date2 = '20151105'

# Locate the specific row with the given 'uid' value and update the 'reported_date'
homicide_data.loc[homicide_data['uid'] == chosen_uid2, 'reported_date'] = new_reported_date2

In [7]:
homicide_data['state'].unique()

array(['NM', 'GA', 'MD', 'LA', 'AL', 'MA', 'NY', 'NC', 'IL', 'OH', 'TX',
       'CO', 'MI', 'CA', 'IN', 'FL', 'MO', 'NV', 'KY', 'TN', 'wI', 'MN',
       'OK', 'NE', 'PA', 'AZ', 'VA', 'DC'], dtype=object)

In [8]:
#Wisconsin is WI not wI
homicide_data['state'] = homicide_data['state'].replace('wI', 'WI')

In [9]:
# Replace "unknown" values with NaN
homicide_data.replace('Unknown', pd.NA, inplace=True)

In [10]:
#Some states/cities did not disclose victim names
homicide_data =  homicide_data.drop(columns = ["victim_last", "victim_first"])

In [11]:
homicide_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52179 entries, 0 to 52178
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   uid            52179 non-null  object 
 1   reported_date  52179 non-null  object 
 2   victim_race    47980 non-null  object 
 3   victim_age     49180 non-null  object 
 4   victim_sex     47948 non-null  object 
 5   city           52179 non-null  object 
 6   state          52179 non-null  object 
 7   lat            52119 non-null  float64
 8   lon            52119 non-null  float64
 9   disposition    52179 non-null  object 
dtypes: float64(2), object(8)
memory usage: 4.0+ MB


In [12]:
homicide_data.disposition = homicide_data.disposition.replace("Closed without arrest", "No Arrest")
homicide_data.disposition = homicide_data.disposition.replace("Open/No arrest", "No Arrest")
homicide_data.disposition = homicide_data.disposition.replace("Closed by arrest", "Arrest Made")

In [13]:
homicide_data['reported_date']

0        20100504
1        20100216
2        20100601
3        20100101
4        20100102
           ...   
52174    20160908
52175    20160913
52176    20161114
52177    20161130
52178    20160901
Name: reported_date, Length: 52179, dtype: object

In [14]:
# Convert the 'reported_date' column to datetime
homicide_data['reported_date'] = pd.to_datetime(homicide_data['reported_date'], format='%Y%m%d')

In [15]:
homicide_data['reported_year'] = homicide_data['reported_date'].dt.year
homicide_data['reported_month'] = homicide_data['reported_date'].dt.strftime('%B')
homicide_data['reported_weekday'] = homicide_data['reported_date'].dt.day_name()

In [16]:
homicide_data['reported_weekday'].value_counts()

Sunday       8606
Saturday     8348
Monday       7564
Friday       7095
Tuesday      6972
Wednesday    6871
Thursday     6723
Name: reported_weekday, dtype: int64

In [17]:
# Convert 'victim_age' to numeric
homicide_data['victim_age'] = pd.to_numeric(homicide_data['victim_age'], errors='coerce')

# Define the age bins and labels
age_bins = [0, 18, 30, 45, 64, float('inf')]
age_labels = ['0-17', '18-29', '30-44', '45-64', '65+']

# Bin the 'victim_age' column and create a new column 'age_range'
homicide_data['age_range'] = pd.cut(homicide_data['victim_age'], bins=age_bins, labels=age_labels, right=False)

In [18]:
homicide_data.age_range.value_counts()

18-29    22181
30-44    13939
45-64     7415
0-17      4060
65+       1585
Name: age_range, dtype: int64

In [19]:
homicide_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52179 entries, 0 to 52178
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   uid               52179 non-null  object        
 1   reported_date     52179 non-null  datetime64[ns]
 2   victim_race       47980 non-null  object        
 3   victim_age        49180 non-null  float64       
 4   victim_sex        47948 non-null  object        
 5   city              52179 non-null  object        
 6   state             52179 non-null  object        
 7   lat               52119 non-null  float64       
 8   lon               52119 non-null  float64       
 9   disposition       52179 non-null  object        
 10  reported_year     52179 non-null  int64         
 11  reported_month    52179 non-null  object        
 12  reported_weekday  52179 non-null  object        
 13  age_range         49180 non-null  category      
dtypes: category(1), dateti

In [20]:
# Define a function to map month values to seasons
def get_season(reported_month):
    if reported_month in ['December','January', 'February']:
        return 'Winter'
    elif reported_month in ['March', 'April', 'May']:
        return 'Spring'
    elif reported_month in ['June', 'July', 'August']:
        return 'Summer'
    elif reported_month in ['September', 'October', 'November']:
        return 'Fall'
    else:
        return 'Unknown'  # Handle invalid month values if any

# Apply the function to create a new "season" column
homicide_data['season'] = homicide_data['reported_month'].apply(get_season)

In [21]:
homicide_data.head()

,uid,reported_date,victim_race,victim_age,victim_sex,city,state,lat,lon,disposition,reported_year,reported_month,reported_weekday,age_range,season
0,Alb-000001,2010-05-04,Hispanic,78.0,Male,Albuquerque,NM,35.095788,-106.538555,No Arrest,2010,May,Tuesday,65+,Spring
1,Alb-000002,2010-02-16,Hispanic,17.0,Male,Albuquerque,NM,35.056810,-106.715321,Arrest Made,2010,February,Tuesday,0-17,Winter
2,Alb-000003,2010-06-01,White,15.0,Female,Albuquerque,NM,35.086092,-106.695568,No Arrest,2010,June,Tuesday,0-17,Summer
3,Alb-000004,2010-01-01,Hispanic,32.0,Male,Albuquerque,NM,35.078493,-106.556094,Arrest Made,2010,January,Friday,30-44,Winter
4,Alb-000005,2010-01-02,White,72.0,Female,Albuquerque,NM,35.130357,-106.580986,No Arrest,2010,January,Saturday,65+,Winter


In [22]:
# Create a new column 'LOCATION' by concatenating 'city' and 'state' with a comma separator
homicide_data['LOCATION'] = homicide_data['city'] + ', ' + homicide_data['state']

In [23]:
homicide_data['LOCATION'].unique()

array(['Albuquerque, NM', 'Atlanta, GA', 'Baltimore, MD',
       'Baton Rouge, LA', 'Birmingham, AL', 'Boston, MA', 'Buffalo, NY',
       'Charlotte, NC', 'Chicago, IL', 'Cincinnati, OH', 'Columbus, OH',
       'Dallas, TX', 'Denver, CO', 'Detroit, MI', 'Durham, NC',
       'Fort Worth, TX', 'Fresno, CA', 'Houston, TX', 'Indianapolis, IN',
       'Jacksonville, FL', 'Kansas City, MO', 'Las Vegas, NV',
       'Long Beach, CA', 'Los Angeles, CA', 'Louisville, KY',
       'Memphis, TN', 'Miami, FL', 'Milwaukee, WI', 'Minneapolis, MN',
       'Nashville, TN', 'New Orleans, LA', 'New York, NY', 'Oakland, CA',
       'Oklahoma City, OK', 'Omaha, NE', 'Philadelphia, PA',
       'Phoenix, AZ', 'Pittsburgh, PA', 'Richmond, VA', 'San Antonio, TX',
       'Sacramento, CA', 'Savannah, GA', 'San Bernardino, CA',
       'San Diego, CA', 'San Francisco, CA', 'St. Louis, MO',
       'Stockton, CA', 'Tampa, FL', 'Tulsa, OK', 'Tulsa, AL',
       'Washington, DC'], dtype=object)

In [24]:
#There is no Tulsa, AL. The lon and lat indicate Oklahoma.
# Replace 'AL' with 'OK' for 'LOCATION' containing 'city' Tulsa.
homicide_data['LOCATION'] = homicide_data['LOCATION'].str.replace(r'Tulsa.*, AL$', 'Tulsa, OK', regex=True)

In [25]:
homicide_data['LOCATION'].unique()

array(['Albuquerque, NM', 'Atlanta, GA', 'Baltimore, MD',
       'Baton Rouge, LA', 'Birmingham, AL', 'Boston, MA', 'Buffalo, NY',
       'Charlotte, NC', 'Chicago, IL', 'Cincinnati, OH', 'Columbus, OH',
       'Dallas, TX', 'Denver, CO', 'Detroit, MI', 'Durham, NC',
       'Fort Worth, TX', 'Fresno, CA', 'Houston, TX', 'Indianapolis, IN',
       'Jacksonville, FL', 'Kansas City, MO', 'Las Vegas, NV',
       'Long Beach, CA', 'Los Angeles, CA', 'Louisville, KY',
       'Memphis, TN', 'Miami, FL', 'Milwaukee, WI', 'Minneapolis, MN',
       'Nashville, TN', 'New Orleans, LA', 'New York, NY', 'Oakland, CA',
       'Oklahoma City, OK', 'Omaha, NE', 'Philadelphia, PA',
       'Phoenix, AZ', 'Pittsburgh, PA', 'Richmond, VA', 'San Antonio, TX',
       'Sacramento, CA', 'Savannah, GA', 'San Bernardino, CA',
       'San Diego, CA', 'San Francisco, CA', 'St. Louis, MO',
       'Stockton, CA', 'Tampa, FL', 'Tulsa, OK', 'Washington, DC'],
      dtype=object)

In [26]:
locations_to_keep = homicide_data['LOCATION'].unique()

In [27]:
homicide_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52179 entries, 0 to 52178
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   uid               52179 non-null  object        
 1   reported_date     52179 non-null  datetime64[ns]
 2   victim_race       47980 non-null  object        
 3   victim_age        49180 non-null  float64       
 4   victim_sex        47948 non-null  object        
 5   city              52179 non-null  object        
 6   state             52179 non-null  object        
 7   lat               52119 non-null  float64       
 8   lon               52119 non-null  float64       
 9   disposition       52179 non-null  object        
 10  reported_year     52179 non-null  int64         
 11  reported_month    52179 non-null  object        
 12  reported_weekday  52179 non-null  object        
 13  age_range         49180 non-null  category      
 14  season            5217

***Population Data***

In [28]:
# Define the file paths using Path
all_pop_csv = Path("data/all_pop_data.csv")

# Load filtered_pop_data.csv into a DataFrame with the specified encoding
all_pop_df = pd.read_csv(all_pop_csv, encoding='latin1')


***Create a new table for specific date***

In [29]:
all_pop_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39274 entries, 0 to 39273
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   NAME      39274 non-null  object
 1   STNAME    39274 non-null  object
 2   LOCATION  39274 non-null  object
 3   2007      39274 non-null  int64 
 4   2008      39274 non-null  int64 
 5   2009      39274 non-null  int64 
 6   2010      39274 non-null  object
 7   2011      39274 non-null  int64 
 8   2012      39274 non-null  int64 
 9   2013      39274 non-null  int64 
 10  2014      39274 non-null  int64 
 11  2015      39274 non-null  int64 
 12  2016      39274 non-null  int64 
 13  2017      39274 non-null  int64 
dtypes: int64(10), object(4)
memory usage: 4.2+ MB


In [30]:
filtered_pop_df = all_pop_df[all_pop_df['LOCATION'].isin(locations_to_keep)]
filtered_pop_df.head()

,NAME,STNAME,LOCATION,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
42,Birmingham,AL,"Birmingham, AL",218880,216505,214394,212237,211554,210775,211270,211179,211811,211512,211125
840,Phoenix,AZ,"Phoenix, AZ",1450901,1456729,1452679,1445632,1469770,1499250,1526505,1555485,1583760,1612303,1633704
1708,Fresno,CA,"Fresno, CA",477659,484443,490262,494665,501623,505554,508993,514052,517408,520305,523938
1791,Long Beach,CA,"Long Beach, CA",460328,460643,461782,462257,464274,466526,468017,468785,469408,468484,466265
1796,Los Angeles,CA,"Los Angeles, CA",3751872,3763566,3781938,3792621,3818812,3847857,3877721,3904102,3933644,3957520,3975067


In [31]:
id_vars = ['NAME', 'STNAME', 'LOCATION']
value_vars = ['2007', '2008', '2009', '2010', '2011',
       '2012', '2013', '2014', '2015', '2016', '2017']

In [32]:
df_melt = pd.melt(filtered_pop_df,id_vars=id_vars,value_vars=value_vars,var_name="YEAR", value_name="POPULATION")
df_melt['YEAR'] = df_melt.YEAR.astype(int)
df_melt['POPULATION'] = df_melt.POPULATION.astype(int)

In [33]:
df_melt.head()

,NAME,STNAME,LOCATION,YEAR,POPULATION
0,Birmingham,AL,"Birmingham, AL",2007,218880
1,Phoenix,AZ,"Phoenix, AZ",2007,1450901
2,Fresno,CA,"Fresno, CA",2007,477659
3,Long Beach,CA,"Long Beach, CA",2007,460328
4,Los Angeles,CA,"Los Angeles, CA",2007,3751872


In [34]:
df_final = pd.merge(homicide_data, df_melt, left_on = ['LOCATION','reported_year'],
                    right_on = ['LOCATION', 'YEAR'], how = 'left')

In [35]:
# Select and retain only the desired columns
final_columns_to_keep = ['uid', 'disposition','victim_sex','victim_race', 'victim_age', 'age_range',
                         'reported_date', 'reported_year','reported_month', 'reported_weekday', 'season',
                         'city', 'state', 'lat', 'lon', 'LOCATION', 'POPULATION']
df_final = df_final[final_columns_to_keep]
df_final.head()

,uid,disposition,victim_sex,victim_race,victim_age,age_range,reported_date,reported_year,reported_month,reported_weekday,season,city,state,lat,lon,LOCATION,POPULATION
0,Alb-000001,No Arrest,Male,Hispanic,78.0,65+,2010-05-04,2010,May,Tuesday,Spring,Albuquerque,NM,35.095788,-106.538555,"Albuquerque, NM",545852
1,Alb-000002,Arrest Made,Male,Hispanic,17.0,0-17,2010-02-16,2010,February,Tuesday,Winter,Albuquerque,NM,35.056810,-106.715321,"Albuquerque, NM",545852
2,Alb-000003,No Arrest,Female,White,15.0,0-17,2010-06-01,2010,June,Tuesday,Summer,Albuquerque,NM,35.086092,-106.695568,"Albuquerque, NM",545852
3,Alb-000004,Arrest Made,Male,Hispanic,32.0,30-44,2010-01-01,2010,January,Friday,Winter,Albuquerque,NM,35.078493,-106.556094,"Albuquerque, NM",545852
4,Alb-000005,No Arrest,Female,White,72.0,65+,2010-01-02,2010,January,Saturday,Winter,Albuquerque,NM,35.130357,-106.580986,"Albuquerque, NM",545852


In [36]:
# Specify the path for the new CSV file where you want to save the filtered data
output_csv = Path("data/tbl_no_drop_homicide_data.csv")

# Save the filtered DataFrame to the new CSV file
df_final.to_csv(output_csv, index=False, header=True)

In [37]:
df_per = df_final.groupby(["LOCATION","reported_year"]).agg({"uid":"size","POPULATION": 'max'}).reset_index()
df_per["homicides_per_100000"] = df_per["uid"]/df_per["POPULATION"]*100000

df_per.head()

,LOCATION,reported_year,uid,POPULATION,homicides_per_100000
0,"Albuquerque, NM",2010,45,545852,8.243993
1,"Albuquerque, NM",2011,38,552105,6.882749
2,"Albuquerque, NM",2012,46,555074,8.287183
3,"Albuquerque, NM",2013,33,557547,5.918784
4,"Albuquerque, NM",2014,28,557566,5.021827


In [38]:
# Specify the path for the new CSV file where you want to save the filtered data
output_csv = Path("data/tbl_homicide_count_per_100000.csv")

# Save the filtered DataFrame to the new CSV file
df_per.to_csv(output_csv, index=False, header=True)